In [1]:
from vespa.package import ApplicationPackage, Schema, Document, Field, RankProfile, HNSW
from vespa.application import Vespa
from vespa.deployment import VespaDocker
import numpy as np
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
import time
import pickle
import os

c:\Users\ramyr\miniconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# 2. Configuração do modelo de embedding (Semantic Search)
embedding_model = SentenceTransformer("all-MiniLM-L6-v2") 

In [3]:
# 3. Definição do App

app_package= ApplicationPackage(name="vespasearch1")

app_package.schema.add_fields(
        Field(name="id", type="string", indexing=["summary", "attribute"]),
        Field(name="text", type="string", indexing=["index", "summary"]),
        Field(name="embedding", type="tensor<float>(d[384])", indexing=["attribute", "summary", "index"],
              ann=HNSW(distance_metric="angular")
        #       ann="true",
        #       attribute={"distance-metric": "euclidean"
                                #      , "method": "hnsw"
                                #      }
        ), 
)

app_package.schema.add_rank_profile(
        RankProfile(name="bm25", first_phase="nativeRank(text)"),
)

app_package.schema.add_rank_profile(
        RankProfile(name="dot_product", first_phase="closeness(embedding)"),
)

app_package.schema.add_rank_profile(
        RankProfile(name="hybrid", first_phase="nativeRank(text) + closeness(embedding)")
)

In [4]:
app_package.schema.inputs = [("query_embedding", "tensor<float>(d[384])")]

In [5]:
print(app_package.schema.schema_to_text)

schema vespasearch1 {
    document vespasearch1 {
        field id type string {
            indexing: summary | attribute
        }
        field text type string {
            indexing: index | summary
        }
        field embedding type tensor<float>(d[384]) {
            indexing: attribute | summary | index
            attribute {
                distance-metric: angular
            }
            index {
                hnsw {
                    max-links-per-node: 16
                    neighbors-to-explore-at-insert: 200
                }
            }
        }
    }
    rank-profile bm25 {
        first-phase {
            expression {
                nativeRank(text)
            }
        }
    }
    rank-profile dot_product {
        first-phase {
            expression {
                closeness(embedding)
            }
        }
    }
    rank-profile hybrid {
        first-phase {
            expression {
                nativeRank(text) + closeness(embedding)
      

In [6]:
# 4. Criando o app e subindo via Docker
# app_package = ApplicationPackage(name="vespasearch1", schema=None)
vespa_docker = VespaDocker(port=8080, container_memory="8G")
app = vespa_docker.deploy(application_package=app_package)

Waiting for configuration server, 0/60 seconds...
Waiting for configuration server, 5/60 seconds...
Waiting for configuration server, 10/60 seconds...
Waiting for configuration server, 15/60 seconds...
Waiting for configuration server, 20/60 seconds...
Waiting for configuration server, 25/60 seconds...
Waiting for configuration server, 30/60 seconds...
Waiting for configuration server, 35/60 seconds...
Waiting for application to come up, 0/300 seconds.
Waiting for application to come up, 5/300 seconds.
Waiting for application to come up, 10/300 seconds.
Waiting for application to come up, 15/300 seconds.
Waiting for application to come up, 20/300 seconds.
Waiting for application to come up, 25/300 seconds.
Waiting for application to come up, 30/300 seconds.
Waiting for application to come up, 35/300 seconds.
Waiting for application to come up, 40/300 seconds.
Waiting for application to come up, 45/300 seconds.
Waiting for application to come up, 50/300 seconds.
Application is up!
Finis

In [7]:
# 5. Indexação de documentos
from tqdm import tqdm
def index_documents(app, docs):
    for doc in tqdm(docs, desc="Indexing documents", unit="document"):
        doc_id = doc["id"]
        text = doc["text"]
        embedding = embedding_model.encode(text).tolist()
        app.feed_data_point(
            schema="vespasearch1",
            data_id=doc_id,
            fields={"id": doc_id, "text": text, "embedding": embedding}
        )

In [8]:
# 6. Funções de Busca
def lexical_search(app, query):
    return app.query(
        yql="select * from sources * where userQuery();",
        query=query,
        ranking="bm25"
    )

def semantic_search(app, query):
    query_vec = embedding_model.encode(query).tolist()
    return app.query(
        yql="select * from sources * where ([{\"targetNumHits\":10}]nearestNeighbor(embedding, query_embedding));",
        query_tensor={"query_embedding": query_vec},
        ranking="dot_product"
    )

def hybrid_search(app, query):
    query_vec = embedding_model.encode(query).tolist()
    return app.query(
        body={
            "yql": """
                select * from sources * where 
                userQuery() or ([{"targetNumHits":10}]nearestNeighbor(embedding, query_embedding));
            """,
            "query": query,
            "ranking": "hybrid",
            "hits": 10,
            "input.query(query_embedding)": query_vec
        }
    )

In [9]:
PATH = 'subset_msmarco_train_0.01_9.pkl'
PATH_DATA = '../data/' + PATH
PATH_DATA_CLEAN = '../data_clean/' + PATH

In [10]:
# 7 Carregando e formatando dados do MSMARCO
with open("../subset_msmarco_train_0/subset_msmarco_train_0.01_9.pkl", "rb") as f:
    data = pickle.load(f)

# Extraindo lista de documentos
docs_raw = data["docs"]
docs = [{"id": doc_id, "text": doc_obj.text} for doc_id, doc_obj in docs_raw.items()]

In [11]:
response = app.get_application_status()
print(response.status_code)
print(response.json())


200
{'application': {'vespa': {'version': '8.520.18'}, 'meta': {'name': 'default', 'user': 'unknown', 'path': '', 'generation': 12, 'timestamp': 1747691043909, 'date': 'Mon May 19 21:44:03 UTC 2025', 'checksum': '4aea6142f0514374a0ca30b13504bc6b'}, 'user': {'version': ''}}, 'abstractComponents': [{'id': 'com.yahoo.search.searchers.InputCheckingSearcher@vespasearch1_content', 'class': 'com.yahoo.search.searchers.InputCheckingSearcher', 'bundle': 'container-search-and-docproc:8.520.18'}, {'id': 'com.yahoo.container.logging.FileConnectionLog', 'class': 'com.yahoo.container.logging.FileConnectionLog', 'bundle': 'container-disc:8.520.18'}, {'id': 'com.yahoo.container.jdisc.state.StateMonitor', 'class': 'com.yahoo.container.jdisc.state.StateMonitor', 'bundle': 'container-disc:8.520.18'}, {'id': 'com.yahoo.search.searchers.OpportunisticWeakAndSearcher@vespa', 'class': 'com.yahoo.search.searchers.OpportunisticWeakAndSearcher', 'bundle': 'container-search-and-docproc:8.520.18'}, {'id': 'com.yah

In [12]:
import requests

r = requests.get("http://localhost:8080/document/v1/document/document/docid/0")
print(r.status_code, r.text)

404 {"pathId":"/document/v1/document/document/docid/0","id":"id:document:document::0","message":"[UNKNOWN(251001) @ tcp/vespasearch1:19115/default]: Unknown bucket space mapping for document type 'document' in id: 'id:document:document::0' "}


In [13]:
import os
print("Local esperado:", os.path.abspath("schemas/document.sd"))
print("Existe?", os.path.exists("schemas/document.sd"))

with open("schemas/document.sd", "r") as f:
    print("Conteúdo:\n")
    print(f.read())

Local esperado: c:\Users\ramyr\OneDrive\Área de Trabalho\FGV\5periodo\ProjectsInDataScience\data-sci-project\src\schemas\document.sd
Existe? True
Conteúdo:

schema document {
    document document {
        field id type string {
            indexing: summary | attribute
        }

        field text type string {
            indexing: index | summary
        }

        field embedding type tensor<float>(d[384]) {
            indexing: attribute | summary
            attribute {
                distance-metric: dotproduct
            }
        }
    }

    rank-profile bm25 inherits default {
        first-phase {
            expression: nativeRank(text)
        }
    }

    rank-profile dot_product inherits default {
        first-phase {
            expression: closeness(embedding)
        }
    }

    rank-profile hybrid inherits default {
        first-phase {
            expression: nativeRank(text) + closeness(embedding)
        }
    }
}


In [ ]:
index_documents(app, docs)

Indexing documents:   6%|▌         | 1572/27778 [02:51<1:33:11,  4.69document/s] 

In [ ]:
hybrid_search(app, "What is the capital of France?")


VespaError: [{'code': 4, 'summary': 'Invalid query parameter', 'source': 'vespasearch1_content', 'message': "Expected 'query(query_embedding)' to be a tensor, but it is the string '[0.08204811066389084, 0.03605553135275841, -0.0038928852882236242, -0.0048810457810759544, 0.02565113641321659, -0.05714348703622818, 0.012191606685519218, 0.004678904078900814, 0.03494987264275551, -0.0224219411611557, -0.008005237206816673, -0.10935354232788086, 0.022724784910678864, -0.02932087890803814, -0.04352205619215965, -0.12024123221635818, -0.000848641328047961, -0.018150122836232185, 0.056129537522792816, 0.003085229778662324, 0.0023363472428172827, -0.01683923974633217, 0.06362469494342804, -0.023660214617848396, 0.03149356320500374, -0.034797921776771545, -0.0205488633364439, -0.002790951170027256, -0.011037975549697876, -0.03612672537565231, 0.0541410930454731, -0.036617133766412735, -0.02500864863395691, -0.03817041590809822, -0.04960364103317261, -0.015148096717894077, 0.02131503075361252, -